# Verification Circuit Synthesis

Example notebook for the verification circuit synthesis task.

In [ ]:
import sys
sys.path.append(r'../')

%load_ext autoreload
%autoreload 2
%matplotlib inline

## 01 - Simplest Example (CSS code)

In the simplest example, you only need to give the encoding circuit. It supports `stim.Circuit` or `qiskit.QuantumCircuit` object

By default, the gate set will be [CX, CZ, H] and use all-to-all qubit connectivity.

We first show an example with `stim.Circuit`.

In [ ]:
import stim
from rlftqc.verification_circuit_synthesis import VerificationCircuitSynthesis

## Define the encoding circuit
## For example, circuit that prepares zero logical of 7 qubit Steane code.
circ = stim.Circuit("""
H 0 1 3
CX 0 6 1 5 0 4 3 4 3 5 5 6 0 2 1 2
""")

## Create class and we can ignore Z error since we are preparing zero logical of a CSS code.
vcs = VerificationCircuitSynthesis(circ, ignore_z_errors = True)


We now train the RL agent. This will train 10 agents in parallel. It takes around 80 seconds to train. 

In [ ]:
vcs.train()

After the training is done, we can now the run the agent to get the circuit.

In [ ]:
vcs.run()

One can also customize the folder name to save the circuits

In [ ]:
vcs.run(results_folder_name='results')

We can also log the result to check the training convergence.

In [ ]:
vcs.log()

We can also customize the folder name to log the experiment.

In [ ]:
## One can also customize the folder name to save log
vcs.log(results_folder_name='logs')

One could also do the same with `qiskit.QuantumCircuit`

In [ ]:
from qiskit import QuantumCircuit
from rlftqc.verification_circuit_synthesis import VerificationCircuitSynthesis

## Define the encoding circuit
## For example, circuit that prepares zero logical of 7 qubit Steane code.
circ = QuantumCircuit(7)
circ.h(0)
circ.h(1)
circ.h(3)
circ.cx(0,6)
circ.cx(1,5)
circ.cx(0,4)
circ.cx(3,4)
circ.cx(3,5)
circ.cx(5,6)
circ.cx(0,2)
circ.cx(1,2)

## Create class and we can ignore Z error since we are preparing zero logical of a CSS code.
vcs = VerificationCircuitSynthesis(circ, ignore_z_errors = True)

## Train the agent
vcs.train()

## Run the agent and get the prepared circuit
vcs.run()

## 02 - Advanced Example (non-CSS code)

We showed an example of synthesizing verification circuit for non-CSS code particularly the $|-\rangle_L$ of the $[[5,1,3]]$ perfect code.

In [ ]:
import stim
from rlftqc.verification_circuit_synthesis import VerificationCircuitSynthesis
from rlftqc.simulators.clifford_gates import CliffordGates

## Define the encoding circuit
## For example, circuit that prepares minus logical of 5 qubit perfect code.
circ = stim.Circuit("""
H 0 1 2 3 4
CZ 0 1 2 3 1 2 3 4 0 4
""")

We will use two flag qubits.

Here, we will also show how to introduce inductive biases to help agent converge faster.

For example, we know that we need to measure in the X basis in the ancilla. Therefore, we can specify it to the VCS task. We also can specify the agent to use only CX and CZ gates. Furthermore, no gates between ancilla is needed.

In [ ]:

cliff_gates = CliffordGates(7)
gates = [cliff_gates.cx, cliff_gates.cz]
plus_ancilla_position = [5,6]

vcs = VerificationCircuitSynthesis(circ, num_ancillas = 2, gates=gates, plus_ancilla_position = plus_ancilla_position, gates_between_ancilla = False, use_max_reward = False)
## Need to change training config such that the agent explore more.
vcs.training_config["TOTAL_TIMESTEPS"] = 1e6
vcs.training_config["LR"] = 5e-4
vcs.training_config["ENT_COEF"] = 0.1

In [ ]:
## Train the agent
vcs.train()

In [ ]:
## Run the agent and get the prepared circuit
vcs.run()

In [ ]:
## Log the training process
vcs.log()